## Crypto and Machine Learning

In [109]:
starting_date = '2017-01-01T00:00:00Z'
ending_date = '2022-04-29T00:00:00Z'
pair = 'BTC/USDT'
tf = "1d"
exchange = 'Binance'

to_present = True
pair_id = pair.split('/')[0] + "-" + pair.split('/')[-1]
print("Pair : " + pair_id)

Pair : BTC-USDT


In [110]:
import ccxt
from datetime import datetime as dt
from freqtrade.data.converter import ohlcv_to_dataframe
import pandas as pd
import time

exchange_dict = {'Binance':ccxt.binance(),
                'Kucoin':ccxt.kucoin(),
                'Coinex':ccxt.coinex(),
                'Kraken':ccxt.kraken(),
                'Coinbase':ccxt.coinbase()}
ex = exchange_dict[exchange]

print(ex.iso8601(ex.milliseconds()), 'Loading markets')
#markets = ex.load_markets()
print(ex.iso8601(ex.milliseconds()), 'Markets loaded')

from_timestamp = ex.parse8601(starting_date)
if to_present:
    now = ex.milliseconds()
else:
    now = ex.parse8601(ending_date)
df_list = []
while from_timestamp < now:
    print('Fetching candles starting from', ex.iso8601(from_timestamp))
    try:
        ohlcvs = ex.fetch_ohlcv(pair, tf, from_timestamp)
    except:
        print("Retrying...")
        continue
    time.sleep(ex.rateLimit / 1000)
    if not len(ohlcvs):
        break
    from_timestamp = ohlcvs[-1][0] + ex.parse_timeframe(tf) * 1000
    df_list.append(ohlcv_to_dataframe(ohlcvs, tf, pair=pair, drop_incomplete=False))

2022-04-29T06:31:10.822Z Loading markets
2022-04-29T06:31:10.822Z Markets loaded
Fetching candles starting from 2017-01-01T00:00:00.000Z
Fetching candles starting from 2018-12-30T00:00:00.000Z
Fetching candles starting from 2020-05-13T00:00:00.000Z
Fetching candles starting from 2021-09-25T00:00:00.000Z


In [160]:
df = pd.concat(df_list, axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
print('Starting date :', df['date'].iloc[0])
print('Ending date :', df['date'].iloc[-1])
print('DF is created.')
print('DF size: ', df.shape[0], 'rows')
df.head()

df.columns = ["date", "open_"+pair, "high_"+pair, "low_"+pair, "close_"+pair, "volume_"+pair]
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d%H:%M').dt.strftime('%Y-%m-%d %H:%M')
df.sort_values(by="date", inplace=True)
df.drop("volume_"+pair, inplace=True, axis=1)
df.set_index("date", inplace=True)
df.head()

Starting date : 2017-08-17 00:00:00+00:00
Ending date : 2022-04-29 00:00:00+00:00
DF is created.
DF size:  1717 rows


,open_BTC/USDT,high_BTC/USDT,low_BTC/USDT,close_BTC/USDT
date,,,,
2017-08-17 00:00,4261.48,4485.39,4200.74,4285.08
2017-08-18 00:00,4285.08,4371.52,3938.77,4108.37
2017-08-19 00:00,4108.37,4184.69,3850.00,4139.98
2017-08-20 00:00,4120.98,4211.08,4032.62,4086.29
2017-08-21 00:00,4069.13,4119.62,3911.79,4016.00


In [205]:
# Daily return:
import numpy as np
ret = df["close_"+pair] / df["open_"+pair] - 1
print(ret.describe())
df['ret_'+pair] = ret
ret = ret.values
df.head()

count    1717.000000
mean        0.002193
std         0.041707
min        -0.395053
25%        -0.016096
50%         0.001771
75%         0.019728
max         0.225832
dtype: float64


,open_BTC/USDT,high_BTC/USDT,low_BTC/USDT,close_BTC/USDT,ret_BTC/USDT
date,,,,,
2017-08-17 00:00,4261.48,4485.39,4200.74,4285.08,0.005538
2017-08-18 00:00,4285.08,4371.52,3938.77,4108.37,-0.041238
2017-08-19 00:00,4108.37,4184.69,3850.00,4139.98,0.007694
2017-08-20 00:00,4120.98,4211.08,4032.62,4086.29,-0.008418
2017-08-21 00:00,4069.13,4119.62,3911.79,4016.00,-0.013057


In [211]:
n_dev = 1

def cluster(r, ret=ret):
    if r >= (ret.mean() + n_dev*ret.std()):
        return 1
    if r <= (ret.mean() - n_dev*ret.std()):
        return -1
    return 0

df["clustered"] = 0
df["clustered"] = df['ret_'+pair].apply(cluster)
print(f"Daily Returns clustered: {sum(df['clustered']==1)}big, {sum(df['clustered']==0)}medium, {sum(df['clustered']==-1)}small.")

Daily Returns clustered: 192big, 1336medium, 189small.


In [213]:
df

,open_BTC/USDT,high_BTC/USDT,low_BTC/USDT,close_BTC/USDT,ret_BTC/USDT,clustered
date,,,,,,
2017-08-17 00:00,4261.48,4485.39,4200.74,4285.08,0.005538,0
2017-08-18 00:00,4285.08,4371.52,3938.77,4108.37,-0.041238,-1
2017-08-19 00:00,4108.37,4184.69,3850.00,4139.98,0.007694,0
2017-08-20 00:00,4120.98,4211.08,4032.62,4086.29,-0.008418,0
2017-08-21 00:00,4069.13,4119.62,3911.79,4016.00,-0.013057,0
...,...,...,...,...,...,...
2022-04-25 00:00,39450.12,40616.00,38200.00,40426.08,0.024739,0
2022-04-26 00:00,40426.08,40797.31,37702.26,38112.65,-0.057226,-1
2022-04-27 00:00,38112.64,39474.72,37881.31,39235.72,0.029467,0


In [487]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

n_input = 4 

x_list = []
y_list = []
y_real_list = []  # for real returns instead of -1, 0, 1.
for i in range(df['ret_'+pair].values.shape[0]-n_input):
    x_list.append(df['ret_'+pair].values[i:i+n_input])
    y_list.append(df['clustered'].values[i+n_input])
    y_real_list.append(df['ret_'+pair].values[i+n_input])
x, y, y_real = np.array(x_list), np.array(y_list), np.array(y_real_list) 

x_train, x_test, y_train, y_test = train_test_split(x * 100, y, test_size=0.3, random_state=101)
dummy, dummy_, y_real_train, y_real_test = train_test_split(x, y_real, test_size=0.3, random_state=101)

print(f"Shape of x: {x.shape}, Shape of y: {y.shape}")
print(f"Shape of x_train: {x_train.shape}, Shape of y_train: {y_train.shape}, Shape of y_train_real: {y_real_train.shape}")
print(f"Shape of x_test: {x_test.shape}, Shape of y_test: {y_test.shape}, Shape of y_test_real: {y_real_test.shape}")

Shape of x: (1713, 4), Shape of y: (1713,)
Shape of x_train: (1199, 4), Shape of y_train: (1199,), Shape of y_train_real: (1199,)
Shape of x_test: (514, 4), Shape of y_test: (514,), Shape of y_test_real: (514,)


In [497]:
# Define a simple model:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(4, 8, 4, ), activation='relu'
                     , max_iter=5000, shuffle=False, solver='lbfgs', random_state=101)
model

MLPRegressor(hidden_layer_sizes=(4, 8, 4), max_iter=5000, random_state=101,
             shuffle=False, solver='lbfgs')

In [498]:
model.fit(x_train, y_train)

MLPRegressor(hidden_layer_sizes=(4, 8, 4), max_iter=5000, random_state=101,
             shuffle=False, solver='lbfgs')

In [499]:
# Predict the Train data
n_dev_y = 1.5 # confidence level
y_train_predicted = model.predict(x_train)
y_up = y_train_predicted.mean() + n_dev_y*y_train_predicted.std()
y_down = y_train_predicted.mean() - n_dev_y*y_train_predicted.std()
print(f"From {y_train_predicted.shape[0]} predictions: \
    {sum(y_train_predicted>y_up)} positive \
    and {sum(y_train_predicted<y_down)} Negetive")

true_neg , false_neg, true_pos, false_pos = 0, 0, 0, 0
predicted_return = 0
for i in range(x_train.shape[0]):
    if y_train_predicted[i] > y_up or y_train_predicted[i] < y_down:
        #print(f"y={round(y_real_train[i], 1)} predicted={round(y_train_predicted[i], 1)}")
        if y_train_predicted[i] > y_up:
            predicted_return += y_real_train[i]
            if y_real_train[i] > 0: true_pos += 1 
            elif y_real_train[i] < 0: false_pos += 1
        if y_train_predicted[i] < y_down:
            predicted_return -= y_real_train[i]
            if y_real_train[i] < 0: true_neg += 1 
            elif y_real_train[i] > 0: false_neg += 1
print(f"With dev={n_dev_y} confidence level, "
      , f"{round((sum(y_train_predicted>y_up)+ sum(y_train_predicted<y_down)) / y_train_predicted.shape[0] *100)}"
      , "% of predictions are confident enough")
print(f"Results on train data: {true_pos}TP, {false_pos}FP, {true_neg}TN, {false_neg}FN")
print(f"Accuracy = {round((true_pos + true_neg)/(true_pos + true_neg + false_pos + false_neg) * 100, 1)}%")
print(f"This Strategy returns {round(predicted_return*100)}% Profit.")

From 1199 predictions:     42 positive     and 109 Negetive
With dev=1.5 confidence level,  13 % of predictions are confident enough
Results on train data: 30TP, 12FP, 65TN, 44FN
Accuracy = 62.9%
This Strategy returns 304% Profit.


In [500]:
# Predict the Test data
n_dev_y = 1.5 # confidence level
y_test_predicted = model.predict(x_test)
y_up = y_test_predicted.mean() + n_dev_y*y_test_predicted.std()
y_down = y_test_predicted.mean() - n_dev_y*y_test_predicted.std()
print(f"From {y_test_predicted.shape[0]} predictions: \
    {sum(y_test_predicted>y_up)} positive \
    and {sum(y_test_predicted<y_down)} Negetive")

true_neg , false_neg, true_pos, false_pos = 0, 0, 0, 0
predicted_return = 0
for i in range(x_test.shape[0]):
    if y_test_predicted[i] > y_up or y_test_predicted[i] < y_down:
        #print(f"y={round(y_real_test[i], 1)} predicted={round(y_test_predicted[i], 1)}")
        if y_test_predicted[i] > y_up:
            predicted_return += y_real_test[i]
            if y_real_test[i] > 0:   true_pos += 1 
            elif y_real_test[i] < 0: false_pos += 1
        if y_test_predicted[i] < y_down:
            predicted_return -= y_real_test[i]
            if y_real_test[i] < 0:   true_neg += 1 
            elif y_real_test[i] > 0: false_neg += 1

print(f"With dev={n_dev_y} confidence level, "
      , f"{round((sum(y_test_predicted>y_up)+ sum(y_test_predicted<y_down)) / y_test_predicted.shape[0] *100)}"
      , "% of predictions are confident enough")                
print(f"Results on test data: {true_pos}TP, {false_pos}FP, {true_neg}TN, {false_neg}FN")
print(f"Accuracy = {round((true_pos + true_neg)/(true_pos + true_neg + false_pos + false_neg) * 100, 1)}%")
print(f"This Strategy returns {round(predicted_return*100)}% Profit.")

From 514 predictions:     18 positive     and 35 Negetive
With dev=1.5 confidence level,  10 % of predictions are confident enough
Results on test data: 14TP, 4FP, 22TN, 13FN
Accuracy = 67.9%
This Strategy returns 56% Profit.


### This was not an accurate approach!

In [465]:
# Developed by Reza Alavi(2022)